# 03 - Langchain with vector store

In this lab, we will introduce [Langchain](https://python.langchain.com/docs/get_started/introduction), a framework for developing applications powered by language models and ask question on custom data using a vector store.

Langchain supports Python and Javascript / Typescript. For this lab, we will use Python.

## Setup

We'll use the `pip` tool to install the `langchain` Python package and `quadrant`

In [1]:
pip install langchain openai tiktoken qdrant-client python-dotenv --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.8 MB/s eta 0:00:0000:0100:01
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.225
    Uninstalling langchain-0.0.225:
      Successfully uninstalled langchain-0.0.225
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


We'll start by loading the movies csv file the `AzureOpenAI` specific components from the `langchain` package.
As with all the other labs, we'll need to provide our API key and endpoint details. We'll also provide the name (id) of the model deployment that we want to use.

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
# API_KEY = "<YOUR API KEY>"
# RESOURCE_ENDPOINT = "<YOUR AZURE OPENAI ENDPOINT>" # For example https://<your azure open ai instance>.openai.azure.com/
# DEPLOYMENT_ID = "<YOUR DEPLOYMENT ID>" # For example "text-davinci-003"
load_dotenv()

# Set this to `azure`
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"

First we will load the data from the csv file into a loader

In [7]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='../../extra/data/movies/movies.csv', source_column='original_title', encoding='utf-8', csv_args={'delimiter':',', 'fieldnames': ['id', 'original_language', 'original_title', 'popularity', 'release_date', 'vote_average', 'vote_count', 'genre', 'overview', 'revenue', 'runtime', 'tagline']})
data = loader.load()
data = data[1:20] # reduce dataset if you want
print('Loaded %s movies' % len(data))


Loaded 19 movies


We will be using the OpenAI embedding

In [12]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1) 

from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    deployment_name=os.environ["DEPLOYMENT_ID"],
    model_name="gpt-35-turbo",
)

Next, we'll configure Langchain to use Qdrant as vector store using docker, embedd the loaded documents and store the embeddings in the vector store. Depending on the rate limiting this might take a while.

```
docker run --name qdrant -p 6333:6333 -p 6334:6334 -v "$(pwd)/labs/extra/data/qdrantstorage" qdrant/qdrant
```

In [13]:
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient

url = "http://localhost:6333"
qdrant = Qdrant.from_documents(
    data,
    embeddings,
    url=url,
    prefer_grpc=False,
    collection_name="my_movies",
)


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Get a vector representation of a given input that can be easily consumed by machine learning models and algorithms. Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 10 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Get a vector representation of a given input that can be easily consumed by machine learning models and algorithms. Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 6 seconds. Please go here: https://aka.ms/oai/quotaincrease if y

Now we are going to test the vector store and search for similarity

In [14]:
vectorstore = qdrant

query = "What is the best 80s movie I should look?"
found_docs = vectorstore.similarity_search(query)

print(found_docs[0].metadata['source'])

America: The Motion Picture


Another way would be to search for similar movies but with a more diverse results

In [15]:
retriever = vectorstore.as_retriever(search_type="mmr")

query = "Which movies are about space travel?"
print(retriever.get_relevant_documents(query)[0].metadata['source'])

Evolution


In [16]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

index_creator = VectorstoreIndexCreator(embedding=embeddings)
docsearch = index_creator.from_loaders([loader])

ValueError: Could not import chromadb python package. Please install it with `pip install chromadb`.

Now we are using a QA chain to ask questions about the movies

In [17]:
openai = OpenAI(deployment_id="d1")
chain = RetrievalQA.from_chain_type(llm=openai, chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question", return_source_documents=True)
query = "Do you have a column called popularity?"
response = chain({"question": query})
print(response['result'])
print(response['source_documents'])

query = "What is the movie with the highest popularity?"
response = chain({"question": query})
print(response['result'])
print(response['source_documents'])

WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.


NameError: name 'docsearch' is not defined

Load the vector database from a file and ask the same question again.

In [3]:
#del vectorstore

from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient

from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1) 

client = QdrantClient(url="http://localhost:6333", prefer_grpc=False)
# client = qdrant_client.QdrantClient(path="../../extra/data/quadrant", prefer_grpc=True)
qdrant = Qdrant(client=client, collection_name="my_movies", embeddings=embeddings)

query = "What the best movie about space travel?"
found_docs = qdrant.similarity_search(query)

print(found_docs[0].metadata['source'])

Evolution


Now lets create a retriever to query it

In [4]:
retriever = qdrant.as_retriever(search_type="mmr")

query = "What the best movie about space travel?"
print(retriever.get_relevant_documents(query)[0].metadata['source'])


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Get a vector representation of a given input that can be easily consumed by machine learning models and algorithms. Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 6 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Get a vector representation of a given input that can be easily consumed by machine learning models and algorithms. Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if yo

Evolution
